In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import wandb

In [ ]:
# # Create a pytorch dataset of images
# dataset_path = 'datasets/face_mask_detection'

# # Calculate dataset mean and std for normalization
# def calculate_mean_std(dataset_path):
#     sum_mean = np.zeros(3, dtype=np.float64)
#     sum_std = np.zeros(3, dtype=np.float64)
#     count = 0
#     class_names = ['without_mask', 'with_mask']
    
#     print("Calculating dataset mean and std...")
#     for class_name in class_names:
#         class_dir = os.path.join(dataset_path, class_name)
#         image_paths = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
        
#         # Process images in batches with tqdm
#         for img_path in tqdm(image_paths, desc=f"Processing {class_name}"):
#             # Read image in grayscale first to check if it's valid
#             if not os.path.exists(img_path):
#                 continue
                
#             img = cv2.imread(img_path)
#             if img is None:
#                 continue
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             img = cv2.resize(img, (128, 128))
#             img = img.astype(np.float32) / 255.0
            
#             # Update running statistics
#             img_mean = img.mean(axis=(0, 1))
#             img_std = img.std(axis=(0, 1))
            
#             sum_mean += img_mean
#             sum_std += img_std
#             count += 1
    
#     mean = sum_mean / count
#     std = sum_std / count
    
#     return mean.astype(np.float32), std.astype(np.float32)

# # Calculate mean and std for the dataset
# mean, std = calculate_mean_std(dataset_path)
# print(f"Dataset mean: {mean}")
# print(f"Dataset std: {std}")

Calculating dataset mean and std...


Processing with_mask: 100%|██████████| 2165/2165 [00:04<00:00, 454.78it/s]

Dataset mean: [0.5748376  0.49752444 0.46703878]
Dataset std: [0.25625145 0.24203679 0.23397043]


In [2]:
dataset_path = 'datasets/face_mask_detection'

# Create a pytorch dataset
class FaceMaskDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = ['without_mask', 'with_mask']
        for i, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.root_dir, class_name)
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                self.images.append(image_path)  # Store path instead of loaded image
                self.labels.append(i)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = cv2.imread(image_path)
        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Resize to match training size
        image = cv2.resize(image, (128, 128))
        # Convert to tensor first
        image = image.astype(np.float32) / 255.0
        image = torch.from_numpy(image).permute(2, 0, 1)  # HWC to CHW
        
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label
    
# Create a pytorch dataloader
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Normalize([0.5748376, 0.49752444, 0.46703878], [0.25625145, 0.24203679, 0.23397043])
])

dataset = FaceMaskDataset(dataset_path, transform=transform)

In [3]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=16, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=16, pin_memory=True)

In [4]:
# Create a highly configurable CNN model
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, 
                 padding=1, pool_type='max', pool_size=2, pool_stride=2, 
                 activation='relu', batch_norm=False):
        super(ConvBlock, self).__init__()
        
        # Conv layer
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        
        # Batch normalization (optional)
        self.batch_norm = None
        if batch_norm:
            self.batch_norm = nn.BatchNorm2d(out_channels)
        
        # Activation function
        self.activation_type = activation
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU(0.1)
        elif activation == 'elu':
            self.activation = nn.ELU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        
        # Pooling layer
        self.pool = None
        if pool_type == 'max':
            self.pool = nn.MaxPool2d(pool_size, pool_stride)
        elif pool_type == 'avg':
            self.pool = nn.AvgPool2d(pool_size, pool_stride)

    def forward(self, x):
        x = self.conv(x)
        if self.batch_norm:
            x = self.batch_norm(x)
        x = self.activation(x)
        if self.pool:
            x = self.pool(x)
        return x

class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=2, 
                 conv_channels=[16, 32, 64], # List of channels for each conv layer
                 kernel_sizes=[3, 3, 3],     # Kernel sizes for each conv layer
                 pool_types=['max', 'max', 'max'],   # Pooling type for each layer
                 pool_sizes=[2, 2, 2],       # Pool sizes for each layer
                 activations=['relu', 'relu', 'relu'],  # Activation functions
                 use_batch_norm=False,       # Whether to use batch normalization
                 fc_sizes=[128],             # Fully connected layer sizes
                 dropout_rates=[0.5],        # Dropout rates for fc layers
                 final_pool_size=(4, 4)):    # Final adaptive pooling size
                 
        super(CNN, self).__init__()
        
        # Ensure all parameter lists have the same length as conv_channels
        num_conv_layers = len(conv_channels)
        kernel_sizes = self._extend_param(kernel_sizes, num_conv_layers)
        pool_types = self._extend_param(pool_types, num_conv_layers)
        pool_sizes = self._extend_param(pool_sizes, num_conv_layers)
        activations = self._extend_param(activations, num_conv_layers)
        
        # Create convolutional blocks
        self.conv_blocks = nn.ModuleList()
        current_channels = in_channels
        
        for i in range(num_conv_layers):
            self.conv_blocks.append(
                ConvBlock(
                    in_channels=current_channels,
                    out_channels=conv_channels[i],
                    kernel_size=kernel_sizes[i],
                    pool_type=pool_types[i],
                    pool_size=pool_sizes[i],
                    activation=activations[i],
                    batch_norm=use_batch_norm
                )
            )
            current_channels = conv_channels[i]
        
        # Final adaptive pooling layer
        self.adaptive_pool = nn.AdaptiveAvgPool2d(final_pool_size)
        
        # Calculate the flattened size after conv and pooling
        self.flattened_size = conv_channels[-1] * final_pool_size[0] * final_pool_size[1]
        
        # Create fully connected layers
        self.fc_layers = nn.ModuleList()
        self.dropout_layers = nn.ModuleList()
        
        current_size = self.flattened_size
        for i, fc_size in enumerate(fc_sizes):
            self.fc_layers.append(nn.Linear(current_size, fc_size))
            self.dropout_layers.append(nn.Dropout(dropout_rates[min(i, len(dropout_rates)-1)]))
            current_size = fc_size
            
        # Final classification layer
        self.final_fc = nn.Linear(current_size, num_classes)
        
        # Save configuration for forward pass
        self.final_pool_size = final_pool_size
        self.num_conv_layers = num_conv_layers
        self.fc_sizes = fc_sizes
        self.activations = activations

    def _extend_param(self, param_list, target_length):
        """Extend parameter list to the target length by repeating the last element"""
        if len(param_list) >= target_length:
            return param_list[:target_length]
        else:
            return param_list + [param_list[-1]] * (target_length - len(param_list))

    def forward(self, x):
        # Pass through all convolutional blocks
        for block in self.conv_blocks:
            x = block(x)
        
        # Apply adaptive pooling
        x = self.adaptive_pool(x)
        
        # Flatten
        x = x.view(-1, self.flattened_size)
        
        # Fully connected layers with activation and dropout
        for i, (fc_layer, dropout_layer) in enumerate(zip(self.fc_layers, self.dropout_layers)):
            x = fc_layer(x)
            # Apply the same activation function as specified for the last conv layer
            if self.activations[-1] == 'relu':
                x = torch.relu(x)
            elif self.activations[-1] == 'leaky_relu':
                x = torch.nn.functional.leaky_relu(x, 0.1)
            elif self.activations[-1] == 'elu':
                x = torch.nn.functional.elu(x)
            elif self.activations[-1] == 'tanh':
                x = torch.tanh(x)
            elif self.activations[-1] == 'sigmoid':
                x = torch.sigmoid(x)
            x = dropout_layer(x)
        
        # Final classification layer
        x = self.final_fc(x)
        return x

In [5]:
sweep_config = {
    'method': 'bayes',  # Bayesian optimization, alternatives: 'grid', 'random'
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.01
        },
        'batch_size': {
            'values': [16, 32, 64, 128]
        },
        'conv_channels': {
            'values': [
                [16, 32, 64], 
                [32, 64, 128],
                [16, 32, 64, 128]
            ]
        },
        'activations': {
            'values': [
                ['relu', 'relu', 'relu'],
                ['leaky_relu', 'leaky_relu', 'leaky_relu'],
            ]
        },
        'fc_sizes': {
            'values': [
                [128],
                [256],
                [128, 64],
                [256, 128]
            ]
        },
        'dropout_rates': {
            'values': [
                [0.0],
                [0.3],
                [0.5],
            ]
        },
        'use_batch_norm': {
            'values': [True, False]
        },
        'optimizer': {
            'values': ['adam', 'adamw']
        }
    }
}

In [6]:
def train_model():
    # Initialize a wandb run
    run = wandb.init()
    
    # Access your hyperparameters from the run config
    config = wandb.config
    
    # Create your model with the hyperparameters
    model = CNN(
        in_channels=3,  # Assuming RGB images
        num_classes=2,  # Binary classification (mask/no mask)
        conv_channels=config.conv_channels,
        activations=config.activations,
        fc_sizes=config.fc_sizes,
        dropout_rates=config.dropout_rates,
        use_batch_norm=config.use_batch_norm
    )
    
    # Create datasets and dataloaders
    # (You'll need to replace this with your actual dataset setup)
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    # Set up optimizer
    if config.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == 'adamw':
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    num_epochs = 10  # You can also make this a hyperparameter
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            correct_train += predicted.eq(labels).sum().item()
        
        train_accuracy = correct_train / total_train
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total_val += labels.size(0)
                correct_val += predicted.eq(labels).sum().item()
        
        val_accuracy = correct_val / total_val
        
        # Log metrics to wandb
        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss / len(train_loader),
            'train_accuracy': train_accuracy,
            'val_loss': val_loss / len(val_loader),
            'val_accuracy': val_accuracy
        })
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Acc: {val_accuracy:.4f}")

In [7]:
# Initialize a sweep
sweep_id = wandb.sweep(sweep_config, project='face-mask-detection-vr-mini-project-1', entity='ritishtest1')

# Run the sweep
wandb.agent(sweep_id, function=train_model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: uo9mdr7h
Sweep URL: https://wandb.ai/ritishtest1/face-mask-detection-vr-mini-project-1/sweeps/uo9mdr7h


wandb: Agent Starting Run: e2iqoq5y with config:
wandb: 	activations: ['relu', 'relu', 'relu']
wandb: 	batch_size: 16
wandb: 	conv_channels: [16, 32, 64]
wandb: 	dropout_rates: [0]
wandb: 	fc_sizes: [256]
wandb: 	learning_rate: 0.002516890529200087
wandb: 	optimizer: adamw
wandb: 	use_batch_norm: True
wandb: Currently logged in as: ritishshrirao (ritishtest1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch 1/10, Train Loss: 0.3583, Train Acc: 0.8626, Val Loss: 0.2449, Val Acc: 0.9206


...

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x787be107ef90>> (for post_run_cell), with arguments args (<ExecutionResult object at 787be323a990, execution_count=7 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 787be323a8a0, raw_cell="# Initialize a sweep
sweep_id = wandb.sweep(sweep_.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/ritish/prog/college/VR/mini_project_1/face_mask_classifier.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


Exception in thread Thread-47 (_run_job):


BrokenPipeError: [Errno 32] Broken pipe

Traceback (most recent call last):


  File "/home/ritish/prog/ML/.venv/lib/python3.13/site-packages/ipykernel/iostream.py", line 509, in parent_header
    return self._parent_header.get()
           ~~~~~~~~~~~~~~~~~~~~~~~^^


In [9]:
# Test accuracy on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation accuracy: {correct / total}')

Validation accuracy: 0.8974358974358975
